In [1]:
import os
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datasets import Dataset
from torch.utils.data import DataLoader
from PIL import Image
from tqdm import tqdm
from transformers import SamModel, SamConfig, SamProcessor
import torch

/data3/animesh/vscode/Kaggle/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Paths to the folders containing the patches and labels
data_dir = "/data3/animesh/vscode/Kaggle/dataset"
patches_dir = os.path.join(data_dir, "training_patches")
labels_dir = os.path.join(data_dir, "training_noisy_labels")

# Function to load all images in a directory
def load_images_from_folder(folder):
    images = []
    filenames = os.listdir(folder)
    for filename in filenames:
        if filename.endswith(".png"):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path)#.convert('RGB')  # Converting to RGB for consistency
            img = np.array(img)  # Convert to numpy array
            images.append(img)
    return images, filenames

# Load training patches and noisy labels
patches, patch_filenames = load_images_from_folder(patches_dir)
noisy_labels, label_filenames = load_images_from_folder(labels_dir)

In [3]:
images = np.array(patches)
masks = np.array(noisy_labels)

In [4]:
# Create a list to store the indices of non-empty masks
valid_indices = [i for i, mask in enumerate(masks) if mask.max() != 0]
# Filter the image and mask arrays to keep only the non-empty pairs
filtered_images = images[valid_indices]
filtered_masks = masks[valid_indices]
filtered_patch_filenames = [patch_filenames[i] for i in valid_indices]
filtered_label_filenames = [label_filenames[i] for i in valid_indices]

In [5]:
print("Image shape:", filtered_images.shape) 
print("Mask shape:", filtered_masks.shape)

Image shape: (4836, 256, 256, 3)
Mask shape: (4836, 256, 256)


In [6]:
# Convert the NumPy arrays to Pillow images and store them in a dictionary
dataset_dict = {
    "image": [Image.fromarray(img) for img in filtered_images],
    "label": [Image.fromarray(mask) for mask in filtered_masks],
}

# Create the dataset using the datasets.Dataset class
dataset = Dataset.from_dict(dataset_dict)

In [7]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 4836
})

In [8]:
df = pd.read_csv('/data3/animesh/vscode/Kaggle/IOU_epoch_10_filtered.csv', index_col= 0)
df.head()

,imageid,iou_score
0,1_11_86_0_836.png,0.360032
1,1_12_78_836_1044.png,0.433616
2,0_31_59_418_627.png,0.479887
3,1_19_79_627_836.png,0.736264
4,1_7_95_0_836.png,0.664070


In [10]:
df_list = list(zip(dataset['image'], dataset['label'], df['iou_score']))


In [15]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label, iou_score = self.data[idx]
        # Convert image and label to appropriate PyTorch tensors
        # ...
        return image, label, iou_score

In [16]:
dataset = CustomDataset(df_list) 

In [17]:
dataset[0][0].shape

AttributeError: 'PngImageFile' object has no attribute 'shape'

In [44]:
torch.tensor(filtered_images).permute(0, 3, 1, 2).squeeze().shape

torch.Size([4836, 3, 256, 256])

In [41]:
torch.tensor(filtered_masks).unsqueeze(1).shape

torch.Size([4836, 1, 256, 256])

In [46]:
data = torch.cat([torch.tensor(filtered_images).permute(0, 3, 1, 2).squeeze(), torch.tensor(filtered_masks).unsqueeze(1)], dim=1)
data.shape

torch.Size([4836, 4, 256, 256])

In [47]:
score = torch.tensor(df['iou_score'].values, dtype = torch.float32)
score.shape

torch.Size([4836])

,imageid,iou_score
0,1_11_86_0_836.png,0.360032
1,1_12_78_836_1044.png,0.433616
2,0_31_59_418_627.png,0.479887
3,1_19_79_627_836.png,0.736264
4,1_7_95_0_836.png,0.664070


In [19]:
def InputImageData(images, masks):
    image = images/255.0
    image =  np.array(image, dtype = np.float32)
    image = torch.tensor(image).permute(2, 0, 1).squeeze()

    mask = np.array(masks, dtype = np.float32)
    mask =  torch.tensor(mask).unsqueeze(0)

    data = torch.cat([image, mask], dim=0)

    return data


In [20]:
bucket = []
for i in range(len(filtered_images)):
    image = filtered_images[i]
    mask = filtered_masks[i]
    data = InputImageData(image, mask)
    bucket.append(data)

In [21]:
bucket[0].shape

torch.Size([4, 256, 256])

In [23]:
len(bucket)

4836

In [73]:


class CustomImageScoreDataset(Dataset):
    def __init__(self, image_list, iou_scores_df):
        self.image_list = image_list
        self.iou_scores_df = iou_scores_df

    def __len__(self):
        return len(self.iou_scores_df)

    def __getitem__(self, idx):
        # Get the image (tensor) and IOU score for the given index
        image = self.image_list[idx]  # shape: [4, 256, 256]
        iou_score = self.iou_scores_df[idx]
        
        return image, iou_score


In [74]:
custom_dataset = CustomImageScoreDataset(bucket, score)

In [77]:
# Check the length of the dataset
print(f"Total number of samples in the dataset: {len(custom_dataset)}")
print(f"shape of the image: {custom_dataset[0][0].shape}")


Total number of samples in the dataset: 4836
shape of the image: torch.Size([4, 256, 256])


In [79]:
# Define batch size
batch_size = 32

# Create the DataLoader
data_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=True, num_workers=4)

# Test the DataLoader
for i, (images, iou_score) in enumerate(custom_dataset):
    print(f"Batch {i+1}")
    # print(f"Images shape: {images.shape}")  # Should be [batch_size, 4, 256, 256]
    # print(f"Scores: {scores}")
    break  # Just to check the first batch


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f1e6de60ee0>
Traceback (most recent call last):
  File "/data3/animesh/vscode/Kaggle/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1477, in __del__
    self._shutdown_workers()
  File "/data3/animesh/vscode/Kaggle/venv/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1441, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/data3/animesh/vscode/Kaggle/venv/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/data3/animesh/vscode/Kaggle/venv/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/data3/animesh/vscode/Kaggle/venv/lib/python3.10/multiprocessing/connection.py", line 936, in wait
    ready = selector.select(timeout)
  File "/data3/animesh/vscode/Kaggle/venv/lib/python3.10/selectors.py", line 416, in select
    fd_

In [66]:
train_features, train_labels = next(iter(data_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/data3/animesh/vscode/Kaggle/venv/lib/python3.10/site-packages/torch/utils/data/_utils/worker.py", line 309, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
  File "/data3/animesh/vscode/Kaggle/venv/lib/python3.10/site-packages/torch/utils/data/_utils/fetch.py", line 50, in fetch
    data = self.dataset.__getitems__(possibly_batched_index)
  File "/data3/animesh/vscode/Kaggle/venv/lib/python3.10/site-packages/datasets/arrow_dataset.py", line 2747, in __getitems__
    n_examples = len(batch[next(iter(batch))])
TypeError: only integer tensors of a single element can be converted to an index
